In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import random
import implicit
from scipy.sparse import csr_matrix

In [2]:
readers = pd.read_csv("../data/readers.csv")
readers = readers.rename(columns={"id":"user_id", "art_id":"nzz_id"})
readers.head()

,user_id,nzz_id
0,1,ld.154103
1,1,ld.142559
2,1,1.18331199
3,1,ld.144819
4,1,ld.1293110


In [3]:
print(readers.shape)
readers = readers.drop_duplicates()
print(readers.shape)

(27855, 2)
(27855, 2)


In [4]:
read_counts = readers["user_id"].value_counts(sort=True)
read_counts = read_counts.rename_axis("user_id").reset_index(name="read_count")
# Biorę pod uwagę tylko użytkowników, którzy przeczytali minimum 5 artykułów
min_read_count = 5
read_counts = read_counts[read_counts["read_count"] > min_read_count]

readers = readers[readers["user_id"].isin(read_counts["user_id"])]
readers

,user_id,nzz_id
10,3,1.18306956
11,3,ld.140509
12,3,ld.151615
13,3,ld.152374
14,3,ld.148811
...,...,...
27850,1000,ld.153608
27851,1000,ld.137667
27852,1000,ld.1289804
27853,1000,ld.154369


In [5]:
reader_article_matrix_df = pd.crosstab(
    readers["user_id"], readers["nzz_id"]
).fillna(0)
reader_ids = list(reader_article_matrix_df.index)
article_ids = list(reader_article_matrix_df.columns)
reader_article_matrix = reader_article_matrix_df.to_numpy()
# Type cast do float bo inczej metoda nie obsługuje
reader_article_csr_matrix = csr_matrix(reader_article_matrix).asfptype()

In [51]:
reader_ids[11]

14

In [6]:

#Building the model
model = implicit.als.AlternatingLeastSquares(factors=100, regularization=0.1, iterations=100)
alpha_val = 40
data_conf = (reader_article_csr_matrix * alpha_val).astype('double')
model.fit(reader_article_csr_matrix)

100%|██████████| 100/100 [00:29<00:00,  3.35it/s]


In [7]:
print(reader_article_csr_matrix)

  (0, 824)	1.0
  (0, 1941)	1.0
  (0, 3097)	1.0
  (0, 3613)	1.0
  (0, 4510)	1.0
  (0, 5019)	1.0
  (0, 5372)	1.0
  (0, 6160)	1.0
  (0, 6617)	1.0
  (0, 7497)	1.0
  (0, 8631)	1.0
  (0, 9307)	1.0
  (0, 9596)	1.0
  (0, 9872)	1.0
  (1, 143)	1.0
  (1, 511)	1.0
  (1, 575)	1.0
  (1, 1288)	1.0
  (1, 1360)	1.0
  (1, 1564)	1.0
  (1, 1985)	1.0
  (1, 2153)	1.0
  (1, 2619)	1.0
  (1, 2913)	1.0
  (1, 3045)	1.0
  :	:
  (979, 4796)	1.0
  (979, 5964)	1.0
  (979, 6035)	1.0
  (979, 6254)	1.0
  (979, 6698)	1.0
  (979, 6716)	1.0
  (979, 6726)	1.0
  (979, 7464)	1.0
  (979, 7594)	1.0
  (979, 7606)	1.0
  (979, 7636)	1.0
  (979, 7718)	1.0
  (979, 7837)	1.0
  (979, 8107)	1.0
  (979, 9060)	1.0
  (979, 9286)	1.0
  (979, 9743)	1.0
  (979, 9779)	1.0
  (979, 10239)	1.0
  (979, 10284)	1.0
  (979, 10292)	1.0
  (979, 10364)	1.0
  (979, 10605)	1.0
  (979, 10820)	1.0
  (979, 10855)	1.0


In [48]:
recoms = model.recommend(11,data_conf)
impli = [(article_ids[recom[0]]) for recom in recoms]

In [49]:
recoms

[(769, 0.36915892),
 (89, 0.13033877),
 (318, 0.0996528),
 (953, 0.09276729),
 (293, 0.06411597),
 (229, 0.063978046),
 (50, 0.06285694),
 (923, 0.055196084),
 (102, 0.05233226),
 (549, 0.050839707)]

In [9]:
import sys
sys.path.append('../code')
from cf_model import CFModel
from model_evaluator import ModelEvaluator
from random_model import RandomModel

model_evaluator = ModelEvaluator()

In [10]:
articles = pd.read_csv("../data/articles_cleaned.csv", encoding="utf-8", parse_dates=["pub_date"])
articles.head()

,nzz_id,author,catchline,department,lead_text,pub_date,title,paragraph
0,ld.149648,Claudia Gabriel,Obligationenfonds mit fixer Laufzeit,Finanzen,Die Idee ist gut: Statt einer einzigen Obligat...,2017-03-09 08:01:21.000,Es gibt noch interessante Varianten,Die Idee ist gut: Statt einer einzigen Obligat...
1,1.18145900,Unknown,Fashion Week New York,Panorama,Zum Auftakt der Fashion Week in New York zeige...,2017-04-11 14:00:29.473,Fashion Week New York,NaN
2,ld.138769,Unknown,E-Banking-Ausfall,Finanzen,Seit Sonntag funktioniert das E-Banking der Po...,2017-01-09 13:55:00.000,Postfinance kämpft mit dem System,Seit Sonntag funktioniert das E-Banking der Po...
3,ld.143700,Unknown,Terror in Frankreich,International,Einen Tag nach dem Angriff auf Soldaten beim P...,2017-02-04 12:50:25.000,Louvre nach Macheten-Angriff wieder geöffnet,Einen Tag nach dem Angriff auf Soldaten beim P...
4,ld.149385,Unknown,Unglück in Panama,Panorama,Bei einem Busunglück in Panama sind 17 Persone...,2017-03-06 07:31:21.000,Bus prallt gegen eine Mauer und stürzt in Fluss,Bei einem Busunglück in Panama sind 17 Persone...


In [11]:

cf_recommender_model = CFModel(n_latent_factors=200)
cf_recommender_model.fit(readers,articles)


In [56]:
read_arts = readers[readers["user_id"] == 14]["nzz_id"].tolist()

In [57]:
cf_recommender_model.recommend(14, articles_to_ignore=read_arts, verbose=True)

,recommendation_strength,nzz_id,catchline,paragraph,department,lead_text,pub_date
0,0.279670,ld.146093,Gespräche mit Soldaten,Eine der unpopulärsten und zugleich persönlich...,Meinung,Eine der unpopulärsten und zugleich persönlich...,2017-02-17 04:30:00.000
1,0.254329,ld.142391,Die E-Mail-Debatte,Jacqueline Badran denkt über die Verstaatlichu...,NZZaS,Jacqueline Badran denkt über die Verstaatlichu...,2017-01-29 00:00:00.000
2,0.253935,ld.142508,Schweizer Polarexpedition,Die Antarktis ist wegen der unwirtlichen Bedin...,Wissenschaft,Die Antarktis ist wegen der unwirtlichen Bedin...,2017-02-10 04:30:00.000
3,0.251354,ld.1289159,Social Media,Instagram ist weiter auf Erfolgskurs. Das zum ...,Digital,Der Foto-Boom hält an – immer mehr User nutzen...,2017-04-27 13:39:13.035
4,0.246810,ld.1292704,Relativ geringe Renditeerwartungen,Family-Offices setzen sich bei der Vermögensan...,Finanzen,Family-Offices streuen ihre Gelder breit und v...,2017-05-12 03:30:00.000
5,0.244462,ld.146931,Zukunft der EU,Das Europa der Bürger gibt es bis jetzt überal...,Meinung,Das Europa der Bürger gibt es bis jetzt überal...,2017-02-22 04:30:00.000
6,0.243364,ld.155809,Meine Träume gehören mir,NaN,Gesellschaft,"Erzählen Sie keine Träume, dafür sind sie zu p...",2017-04-07 03:30:00.000
7,0.238026,ld.1288371,"Raphael Wicky hatte den Mut, zu warten",NaN,NZZaS,Der FC Basel geht mit der Verpflichtung von Ra...,2017-04-23 06:49:39.552
8,0.234193,ld.139305,Impfstrategie des Bundes,Der Bundesrat will die Bevölkerung mit bessere...,Meinung,Der Bundesrat will die Bevölkerung mit bessere...,2017-01-11 16:53:49.000
9,0.231041,ld.144202,Falschnachrichten in sozialen Netzwerken,Mit speziellen Tools und Datenanalysen wollen ...,Digital,Mit speziellen Tools und Datenanalysen wollen ...,2017-02-07 17:00:00.000


In [54]:
cf_recommender_model.recommend(14, verbose=True)

,recommendation_strength,nzz_id,catchline,paragraph,department,lead_text,pub_date
0,0.912552,ld.140509,EU-Parlament,Erstmals ist ein Italiener Präsident des EU-Pa...,Video,Erstmals ist ein Italiener Präsident des EU-Pa...,2017-01-18 12:09:44.000
1,0.874783,ld.137069,Jahresrückblick Digital,Im Brettspiel Go konnte ein Computer einen men...,Jahresrückblick 2016,Im Brettspiel Go konnte ein Computer einen men...,2017-01-01 05:00:00.000
2,0.835986,ld.1294212,Computerspiele,Im Internet sind interne Firmen-Dokumente aufg...,Digital,Der japanische Computerspiel-Pionier Sega will...,2017-05-17 12:22:35.351
3,0.780219,ld.1289822,Social Media,Der australischen Tageszeitung «The Australian...,Digital,Laut Medienberichten wertet Facebook die Gefüh...,2017-05-02 12:37:14.404
4,0.736583,ld.155500,Computersicherheit,NaN,Digital,Die Gesichtserkennung des neuen Top-Smartphone...,2017-04-05 09:42:25.624
5,0.714677,ld.139073,Digital Video,Kodak hat anlässlich der Consumer Electronics ...,Digital,Kodak hat anlässlich der Consumer Electronics ...,2017-01-10 17:22:26.000
6,0.710976,ld.140906,Digitalfotografie,Die amerikanische Firma Beastgrip hat Adapter ...,Digital,Die amerikanische Firma Beastgrip hat Adapter ...,2017-01-20 06:42:11.000
7,0.703265,ld.1289953,Asset-Allocation von institutionellen Investoren,Die Tiefzinspolitik der Europäischen Zentralba...,Finanzen,Zur Renditejagd gehen institutionelle Anleger ...,2017-05-03 03:30:00.000
8,0.702097,ld.1288746,Apple,Apple ist ganz überraschend in Spendierlaune. ...,Digital,Apple offeriert mehrere bisher kostenpflichtig...,2017-04-25 14:27:23.063
9,0.692531,ld.1293604,Nachwehen vom 15. Januar 2015,Nach dem Entscheid des Handelsgerichts Zürich ...,Finanzen,Hat das Saxo-Urteil des Handelsgerichts Zürich...,2017-05-16 04:30:00.000


In [34]:
real = readers[readers["user_id"] == 4]["nzz_id"].tolist()
real = set(real)

print(real.intersection(set(cf_rec)))
print(real.intersection(set(impli)))

set()
set()


In [ ]:
print(real)

{'ld.142295', 'ld.150796', 'ld.151615', 'ld.145762', 'ld.148811', '1.18306956', 'ld.139682', 'ld.152374', 'ld.138478', 'ld.1288065', 'ld.143413', 'ld.1292030', 'ld.140509', 'ld.1294037'}


In [53]:
print(impli)
print(cf_rec)

       nzz_id
0  1.18280909
1  1.17380691
2  1.18084881
3  1.18366029
4  1.18069023
5  1.17995230
6  1.16492955
7  1.18354729
8  1.17453399
9  1.18192659
['ld.140509', '1.18306956', 'ld.151615', 'ld.137069', 'ld.148811', 'ld.145762', 'ld.152341', 'ld.1288065', 'ld.1292030', 'ld.143413']


In [52]:
readers[readers["nzz_id"].isin(impli)]
impli = pd.DataFrame(impli, columns=["nzz_id"])
recommendations_df = impli.merge(
         articles, how="left", left_on="nzz_id", right_on="nzz_id"
     )[
         [
             "nzz_id",
             "catchline",
             "paragraph",
             "department",
             "lead_text",
             "pub_date",
         ]
     ]
recommendations_df

,nzz_id,catchline,paragraph,department,lead_text,pub_date
0,1.18280909,Prinz George spielt,NaN,Panorama,Seit Montag ist der acht Monate alte Prinz Geo...,2017-04-11 14:02:47.957
1,1.17380691,Dürre in den USA,NaN,Panorama,Die USA werden derzeit von der schwersten Dürr...,2017-04-11 13:57:29.897
2,1.18084881,Jupp Heynckes,NaN,Sport,Jupp Heynckes verbucht mit Bayern München eine...,2017-04-11 13:59:29.587
3,1.18366029,Ausstellung: Spanische Architekturfotografie i...,NaN,Feuilleton,Eine Ausstellung im Museo ICO in Madrid präsen...,2017-04-11 14:04:25.381
4,1.18069023,50 Jahre Europarat,NaN,Schweiz,Am 6. Mai trat die Schweiz dem Europarat bei. ...,2017-04-11 13:59:17.925
5,1.17995230,Chinesisches Neujahr,NaN,Panorama,Millionen haben in China und Ostasien den Begi...,2017-04-11 13:58:44.565
6,1.16492955,Die Explosion der Hindenburg,NaN,Unknown,Am 6. Mai 1937 ging mit der «Hindenburg»-Katas...,2017-04-11 13:57:03.860
7,1.18354729,Eine Nacht in Zürich,NaN,Zürich,Wer ein ganzes Wochenende fast ohne Schlaf dur...,2017-04-11 14:04:13.413
8,1.17453399,«Curiosity» schickt neue Bilder vom Mars,NaN,Unknown,Der Mars-Rover «Curiosity» verrät mehr über se...,2017-04-11 13:57:34.924
9,1.18192659,Durchmesserlinie verbindet Oerlikon und Zürich,NaN,Schweiz,Vom kommenden Sommer an verbindet ein neuer Tu...,2017-04-11 14:01:16.974
